随机梯度下降



In [12]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.utils import data
from d2l import torch as d2l
import random
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file

X_train, y_train = load_svmlight_file("housing_scale.txt")
y_train = torch.tensor(y_train)
X_train = X_train.todense()
X_train = torch.tensor(X_train)
X_train = X_train.float()
y_train = y_train.float().reshape(-1, 1)
# 分隔数据
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.33, random_state=42)
print(y_test.size())


# print(X_test.size())

# 读取数据
def load_array(data_arrays, batch_size, is_train=True):  # @save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


# 分隔数据

batch_size = 64
num_epochs = 200
lr_train = 0.08
data_iter = load_array((X_train, y_train), batch_size)

net = nn.Sequential(nn.Linear(13, 1))

# 初始化参数
net[0].weight.data.normal_(0, 0.1)
net[0].bias.data.fill_(0)

# 设置损失函数
# 均方误差，返回所有样本的平均误差
loss = nn.MSELoss()

# 定义优化函数
# 小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。
trainer = torch.optim.SGD(net.parameters(), lr=lr_train)

# 训练

# 一共运行三次，每一次都运行完整个数据集
# 每一次都是小批量下降batch_size=10
x_data = []

for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(X_train), y_train)
    print(f'epoch {epoch + 1}, loss_train {l:f}')
    x_data.append(l.detach())

print(net[0].weight)
print(net[0].bias)

l = loss(net(X_test), y_test)
print(f'loss_val :  {l:f}')

d2l.plot(list(range(1, num_epochs + 1)), [x_data], xlabel='epoch', ylabel='rmse', xlim=[1, num_epochs],
         legend=['train'])
plt.show()






[]